In [1]:
# 此ipynb文件的目的是合并train.json, valid.json和生成的系统提示词

In [2]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [3]:
import json

# # 读取train.json
# with open('stage1/data_raw/train.json', 'r') as file:
#     train_data = json.load(file) # list, [{}, {}, ...]

# # 读取valid.json
# with open('stage1/data_raw/valid.json', 'r') as file:
#     valid_data = json.load(file) # list, [{}, {}, ...]
# print(len(valid_data))

# # 读取生成的训练阶段的系统提示词
# with open('stage1/system_prompts_gpt/system_prompts_batch_0.json', 'r') as file: # train_system_prompt.json
#     train_system_prompt = json.load(file) # []

# 读取生成的验证阶段的系统提示词
# with open('stage1/system_prompts_gpt/valid_system_prompt.json', 'r') as file:
#     valid_system_prompt = json.load(file) # []

In [4]:
# len_train_data, len_train_system_prompt = len(train_data), len(train_system_prompt)
# len_valid_data, len_valid_system_prompt = len(valid_data), len(valid_system_prompt)

In [5]:
import sys
import os

# 添加项目根目录到Python路径
project_root = "/home/cuipeng/Gemma"
sys.path.append(project_root)

# 现在可以正常导入src下的模块
from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template, format_system

### 合并train.json和train_system_prompt.json

In [6]:
# import numpy as np

In [7]:
# fixed_system_prompts = [
#     "你是一个有帮助的AI助手。",
#     "作为一个AI助手，你会提供准确、有帮助的回答。",
#     "你是一个知识渊博的AI助手，善于解释复杂概念。",
#     "你是一个友好的AI助手，会用通俗易懂的方式回答问题。",
#     "作为AI助手，你会以专业、客观的方式提供帮助。",
#     "你是一个可靠的AI助手，始终保持耐心和专注。",
#     "作为AI助手，你擅长提供清晰、结构化的解答。",
#     "你是一个认真负责的AI助手，会仔细理解并回答问题。",
#     "作为AI助手，你会以严谨的态度提供准确的信息。",
#     "你是一个智能AI助手，能够理解上下文并给出恰当的回应。"
#     "作为一个AI助理，我会专业、友善地回答你的问题。",
#     "我是一个知识渊博的AI，很高兴能帮助你解决问题。",
#     "让我们一起探讨这个问题，我会尽我所能提供帮助。",
#     "我是一个智能助手，擅长理解和解答各类问题。",
#     "作为你的AI伙伴，我会用清晰简洁的方式回答问题。",
#     "我是一个全能型AI助手，可以处理各种类型的任务。",
#     "作为AI助手，我会以严谨专业的态度回应你的需求。",
#     "我是一个AI助理，会用通俗易懂的方式解释复杂概念。",
#     "作为你的AI搭档，我会用准确和有见地的方式回答问题。"
#     ]

In [8]:
# required_fixed_prompt_num = len_train_data * 0.4

# for i in range(len(train_data)):
#     if i < len_train_system_prompt: # 添加由gpt生成的系统提示词
#         train_data[i]['text'] = format_model(train_system_prompt[i]) + train_data[i]['text']
#     elif i < len_train_system_prompt + required_fixed_prompt_num: # 添加固定的系统提示词
#         train_data[i]['text'] = format_model(np.random.choice(fixed_system_prompts)) + train_data[i]['text']
#     else: # 不添加提示词
#         train_data[i]['text'] = train_data[i]['text']

In [9]:
# # 统计添加的不同类型的系统提示词的数量
# print(f"添加由gpt生成的系统提示词的数量: {len_train_system_prompt} 条，占比: {(len_train_system_prompt / len_train_data) * 100}%")
# print(f"添加固定系统提示词的数量: {required_fixed_prompt_num} 条，占比: {(required_fixed_prompt_num / len_train_data) * 100}%")
# print(f"未添加系统提示词的数量: {len_train_data - len_train_system_prompt - required_fixed_prompt_num} 条，占比: {((len_train_data - len_train_system_prompt - required_fixed_prompt_num) / len_train_data) * 100}%")

In [10]:
# # 保存
# import os

# # 确保目录存在
# os.makedirs('stage1/data_final', exist_ok=True)

# with open('stage1/data_final/train.json', 'w') as file:
#     json.dump(train_data, file, indent=4) # indent=4 表示缩进4个空格

### 合并valid.json和valid_system_prompt.json

In [11]:
# 略

### 参数化之后的方法 - 最终使用的方法

In [12]:
import json
import numpy as np # type: ignore

In [13]:
class DataIntegrator:
    def __init__(
        self,
        train_raw_path: str,
        valid_raw_path: str,
        system_prompts_dir: str,
        output_dir: str,
        prompt_ratio: dict = {"gpt": 0.4, "fixed": 0.4, "none": 0.2}
    ):
        """
        初始化数据整合器
        
        Args:
            train_raw_path: 原始训练数据路径
            valid_raw_path: 原始验证数据路径
            system_prompts_dir: GPT生成的系统提示词目录
            output_dir: 输出目录
            prompt_ratio: 三种提示词的目标比例，和需要为1
        """
        self.train_raw_path = train_raw_path
        self.valid_raw_path = valid_raw_path
        self.system_prompts_dir = system_prompts_dir
        self.output_dir = output_dir
        self.prompt_ratio = prompt_ratio
        
        # 验证比例和是否为1
        if abs(sum(prompt_ratio.values()) - 1) > 1e-6:
            raise ValueError("提示词比例之和必须为1")
        
        # 确保输出目录存在
        os.makedirs(output_dir, exist_ok=True)
        
        # 固定的系统提示词列表
        self.fixed_system_prompts = [
            "你是一个有帮助的AI助手。",
            "作为一个AI助手，你会提供准确、有帮助的回答。",
            "你是一个知识渊博的AI助手，善于解释复杂概念。",
            "你是一个友好的AI助手，会用通俗易懂的方式回答问题。",
            "作为AI助手，你会以专业、客观的方式提供帮助。",
            "你是一个可靠的AI助手，始终保持耐心和专注。",
            "作为AI助手，你擅长提供清晰、结构化的解答。",
            "你是一个认真负责的AI助手，会仔细理解并回答问题。",
            "作为AI助手，你会以严谨的态度提供准确的信息。",
            "你是一个智能AI助手，能够理解上下文并给出恰当的回应。"
            "作为一个AI助理，我会专业、友善地回答你的问题。",
            "我是一个知识渊博的AI，很高兴能帮助你解决问题。",
            "让我们一起探讨这个问题，我会尽我所能提供帮助。",
            "我是一个智能助手，擅长理解和解答各类问题。",
            "作为你的AI伙伴，我会用清晰简洁的方式回答问题。",
            "我是一个全能型AI助手，可以处理各种类型的任务。",
            "作为AI助手，我会以严谨专业的态度回应你的需求。",
            "我是一个AI助理，会用通俗易懂的方式解释复杂概念。",
            "作为你的AI搭档，我会用准确和有见地的方式回答问题。"
            ]

    def load_data(self):
        """加载所有需要的数据"""
        # 加载原始数据
        with open(self.train_raw_path, 'r', encoding='utf-8') as f:
            self.train_data = json.load(f)
        with open(self.valid_raw_path, 'r', encoding='utf-8') as f:
            self.valid_data = json.load(f)
            
        # 加载GPT生成的系统提示词
        self.train_system_prompts = []
        self.valid_system_prompts = []
        
        # 加载训练集系统提示词
        train_prompt_path = os.path.join(self.system_prompts_dir, "train_system_prompt.json")
        if os.path.exists(train_prompt_path):
            with open(train_prompt_path, 'r', encoding='utf-8') as f:
                self.train_system_prompts = json.load(f)
                
        # 加载验证集系统提示词
        valid_prompt_path = os.path.join(self.system_prompts_dir, "valid_system_prompt.json")
        if os.path.exists(valid_prompt_path):
            with open(valid_prompt_path, 'r', encoding='utf-8') as f:
                self.valid_system_prompts = json.load(f)
                
        print("数据加载完成：")
        print(f"- 训练集数据: {len(self.train_data)} 条")
        print(f"- 验证集数据: {len(self.valid_data)} 条")
        print(f"- 训练集GPT生成提示词: {len(self.train_system_prompts)} 条")
        print(f"- 验证集GPT生成提示词: {len(self.valid_system_prompts)} 条")

    def integrate_data(self, data, system_prompts, dataset_type: str):
        """整合数据集
        
        Args:
            data: 原始数据
            system_prompts: 实际GPT生成的系统提示词
            dataset_type: 数据集类型（'train' 或 'valid'）
        """
        total_count = len(data)
        actual_gpt_count = len(system_prompts) # 实际GPT生成的系统提示词数量，也就是“system_prompts_gpt”目录下文件中生成的条数
        target_gpt_count = int(total_count * self.prompt_ratio["gpt"]) # 目标比例 * 总数
        
        print(f"\n{dataset_type} 数据集统计:")
        print(f"总数据量: {total_count}")
        print(f"目标GPT生成提示词数量: {target_gpt_count} ({self.prompt_ratio['gpt']*100:.1f}%)")
        print(f"实际GPT生成提示词数量: {actual_gpt_count}")
        
        # 确定使用的GPT提示词数量
        if actual_gpt_count >= target_gpt_count: # 如果实际数量足够，使用目标数量
            used_gpt_count = target_gpt_count # used_gpt_count: 最终要使用多少个gpt生成的系统提示词
            fixed_count = int(total_count * self.prompt_ratio["fixed"])
            none_count = total_count - used_gpt_count - fixed_count
        else: # 如果实际数量不足，使用全部实际数量，并重新分配剩余比例
            used_gpt_count = actual_gpt_count
            shortage_count = total_count - ((self.prompt_ratio["fixed"] + self.prompt_ratio["none"]) * total_count) - actual_gpt_count # 使用数量而不是比例，因为小数计算可能会出现误差
            # 将未满足的比例分配给fixed
            fixed_count = self.prompt_ratio["fixed"] * total_count + shortage_count
            none_count = total_count - used_gpt_count - fixed_count # 应该就等于 self.prompt_ratio["none"] * total_count
        
        print("\n最终分配:")
        print(f"使用GPT生成提示词数量: {used_gpt_count} ({used_gpt_count/total_count*100:.1f}%)")
        print(f"使用固定提示词数量: {fixed_count} ({fixed_count/total_count*100:.1f}%)")
        print(f"无提示词数量: {none_count} ({none_count/total_count*100:.1f}%)")
        
        # 创建新的数据列表
        processed_data = data.copy()
        
        if actual_gpt_count > target_gpt_count: # 如果实际数量小于目标数量，则截取前“target_gpt_count”个 
            selected_prompts = system_prompts[:target_gpt_count]
        else: # 如果实际数量大于目标数量，则全部使用
            selected_prompts = system_prompts

        # 添加系统提示词
        for i in range(len(processed_data)): # used_gpt_count: 要使用多少个gpt生成的系统提示词
            if i < used_gpt_count:  # 添加GPT生成的提示词
                processed_data[i]['text'] = format_system(selected_prompts[i]) + processed_data[i]['text']
            elif i < used_gpt_count + fixed_count:  # 添加固定提示词
                processed_data[i]['text'] = format_system(np.random.choice(self.fixed_system_prompts)) + processed_data[i]['text']
            # 其余数据保持原样
        
        return processed_data

    def process(self):
        """处理并保存数据"""
        # 加载数据
        self.load_data()
        
        # 处理训练集
        processed_train = self.integrate_data(
            self.train_data,
            self.train_system_prompts,
            'train'
        )
        
        # 处理验证集
        processed_valid = self.integrate_data(
            self.valid_data,
            self.valid_system_prompts,
            'valid'
        )
        
        # 保存处理后的数据
        train_path = os.path.join(self.output_dir, 'train.json')
        with open(train_path, 'w', encoding='utf-8') as f:
            json.dump(processed_train, f, ensure_ascii=False, indent=4)
            
        valid_path = os.path.join(self.output_dir, 'valid.json')
        with open(valid_path, 'w', encoding='utf-8') as f:
            json.dump(processed_valid, f, ensure_ascii=False, indent=4)
        
        print("\n数据保存完成：")
        print(f"- 训练集: {train_path} ({len(processed_train)} 条数据)")
        print(f"- 验证集: {valid_path} ({len(processed_valid)} 条数据)")

In [14]:
# system_prompts_dir='stage1/system_prompts_gpt',

# with open(f'{system_prompts_dir}/train_system_prompt.json', 'r') as file:
#     train_data = json.load(file) # list, [{}, {}, ...]
#     len_train_system_prompt = len(train_data)

# with open(f'{system_prompts_dir}/valid_system_prompt.json', 'r') as file:
#     valid_data = json.load(file) # list, [{}, {}, ...]
#     len_valid_system_prompt = len(valid_data)

# 使用示例
def main():
    integrator = DataIntegrator(
        train_raw_path='stage1/data_raw/train.json',
        valid_raw_path='stage1/data_raw/valid.json',
        system_prompts_dir='stage1/system_prompts_gpt',
        output_dir='stage1/data_final',
        prompt_ratio={"gpt": 0.4, "fixed": 0.4, "none": 0.2}
    )

    integrator.process()

In [15]:
if __name__ == "__main__":
    main()

数据加载完成：
- 训练集数据: 80000 条
- 验证集数据: 1000 条
- 训练集GPT生成提示词: 32500 条
- 验证集GPT生成提示词: 500 条

train 数据集统计:
总数据量: 80000
目标GPT生成提示词数量: 32000 (40.0%)
实际GPT生成提示词数量: 32500

最终分配:
使用GPT生成提示词数量: 32000 (40.0%)
使用固定提示词数量: 32000 (40.0%)
无提示词数量: 16000 (20.0%)

valid 数据集统计:
总数据量: 1000
目标GPT生成提示词数量: 400 (40.0%)
实际GPT生成提示词数量: 500

最终分配:
使用GPT生成提示词数量: 400 (40.0%)
使用固定提示词数量: 400 (40.0%)
无提示词数量: 200 (20.0%)

数据保存完成：
- 训练集: stage1/data_final/train.json (80000 条数据)
- 验证集: stage1/data_final/valid.json (1000 条数据)


In [ ]:
# 生成的数据集的系统提示词是顺序的，前40%是gpt生成的，后40%是固定提示词，最后20%是原数据